# Read the CSV and Perform Basic Data Cleaning

In [24]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [25]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn import preprocessing
from sklearn import utils

In [27]:
df = pd.read_csv(Path('./Machine_Learning/mask.csv', index_col = False))
df.head()

,state_id,date,infection_rate,requirements
0,AK,44256,0.99,0
1,AK,44257,0.99,0
2,AK,44258,0.99,0
3,AK,44259,0.99,0
4,AK,44260,0.99,0


In [28]:
df.dtypes

state_id           object
date                int64
infection_rate    float64
requirements        int64
dtype: object

In [29]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

In [31]:
# remove State
df = df.drop('state_id',1)
df.head()

,date,infection_rate,requirements
0,44256,0.99,0
1,44257,0.99,0
2,44258,0.99,0
3,44259,0.99,0
4,44260,0.99,0


# Split the Data into Training and Testing

In [32]:
# Create our features
X = df.drop("requirements", axis=1)
X = pd.get_dummies(X)

# Create our target
y = df["requirements"]

In [33]:
X.describe()

,date,infection_rate
count,18200.000000,18200.000000
mean,44437.500000,0.999189
std,105.080239,0.196922
min,44256.000000,0.420000
25%,44346.750000,0.890000
50%,44437.500000,0.990000
75%,44528.250000,1.110000
max,44619.000000,1.770000


In [34]:
# Check the balance of our target values
y.value_counts()

0    13468
1     4732
Name: requirements, dtype: int64

In [35]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Ensemble Learners

In [36]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
random_forest = BalancedRandomForestClassifier(n_estimators = 100)
random_forest = random_forest.fit(X_train, y_train)

In [37]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_predicted = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_predicted)

0.5768606138107417

In [38]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predicted)

array([[1859, 1541],
       [ 452,  698]], dtype=int64)

In [39]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predicted))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.80      0.55      0.61      0.65      0.58      0.33      3400
          1       0.31      0.61      0.55      0.41      0.58      0.33      1150

avg / total       0.68      0.56      0.59      0.59      0.58      0.33      4550



In [40]:
# List the features sorted in descending order by feature importance
feature_names = X.columns
sorted(zip(random_forest.feature_importances_, feature_names), reverse=True)

[(0.6568238681292677, 'date'), (0.3431761318707322, 'infection_rate')]

# Easy Ensemble AdaBoost Classifier

In [41]:
# Train the EasyEnsembleClassifier
easy = EasyEnsembleClassifier(n_estimators = 100,random_state=1)
easy = easy.fit(X_train, y_train)

In [42]:
# Calculated the balanced accuracy score
y_predicted = easy.predict(X_test)
balanced_accuracy_score(y_test, y_predicted)

0.5254283887468031

In [43]:
# Display the confusion matrix
confusion_matrix(y_test, y_predicted)

array([[1459, 1941],
       [ 435,  715]], dtype=int64)

In [44]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_predicted))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.77      0.43      0.62      0.55      0.52      0.26      3400
          1       0.27      0.62      0.43      0.38      0.52      0.27      1150

avg / total       0.64      0.48      0.57      0.51      0.52      0.26      4550

